## Supplementary notebook for Mechanical control of tissue growth during limb regeneration.

$Sandra\text{ }Edwards-Jorquera^{1\text{,}2\text{,}*}$, $Alberto\text{ }Ceccarelli^3$, $Raimund\text{ }Schlüßler^{4\text{,}5}$, $Nivedha\text{ }Murali\text{ }Shankar^1$, $Angela\text{ }Muñoz\text{ }Ovalle^1$, $Kyle\text{ }Gentleman^2$, $Uwe\text{ }Freudenberg^{1\text{,}6}$, $Carsten\text{ }Werner^{1\text{,}6}$, $Osvaldo\text{ }Chara^{3\text{,}7\text{,}8}$, $Anna\text{ }Taubenberger^4$, $Tatiana\text{ }Sandoval-Guzmán^{1\text{,}2\text{,}9\text{,}*}$

 

  1.       Center for Regenerative Therapies Dresden (CRTD), TUD Dresden University of Technology, Dresden, Germany.

  2.       Dept. of Internal Medicine III, Center for Healthy Aging, University Hospital Carl Gustav Carus, TUD Dresden University of Technology, Dresden, Germany.

  3.       School of Biosciences, University of Nottingham, Sutton Bonington Campus, Nottingham LE12, U.K.

  4.       Biotechnology Center, Center for Molecular and Cellular Bioengineering (CMCB), TUD Dresden University of Technology, Dresden, Germany.

  5.       CellSense Technologies GmbH, Berlin, Germany.

  6.       ﻿Leibniz Institute of Polymer Research Dresden (IPF), Max Bergmann Center of Biomaterials Dresden (MBC), TUD Dresden University of Technology, Dresden, Germany.

  7.       Center for Information Services and High-Performance Computing, TUD Dresden University of Technology, Dresden, Germany.

  8.       Instituto de Tecnología, Universidad Argentina de la Empresa, Buenos Aires, Argentina.

  9.       Paul Langerhans Institute Dresden (PLID), Helmholtz Centre Munich, University Hospital Carl Gustav Carus, TUD Dresden University of Technology, Dresden, Germany.

* Correspondence: sandra.edwards@tu-dresden.de, tatiana.sandoval_guzman@tu-dresden.de

This notebook contains the code used to fit the mathematical model of limb growth to the experimental data. It reads the Excel file with the data (in this case, the data for 4-month-old animals, change the name of the file to analyze a different experimental set), fits the mathematical expression to it, and saves the parameters obtained from the fit as a .txt file.

In [ ]:
# Import libraries.
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
import random
import copy

# Read the data.

dataset = "upper"
archivo = "datos-growth2-upper-joint.xlsx"
df = pd.read_excel(io=archivo, sheet_name=str(dataset))
area_a1 = df["a1"]
area_a2 = df["a2"]
area_a3 = df["a3"]
time = df["time"]
filenames=["upper_a1","upper_a2","upper_a3"]
animals = [area_a1,area_a2,area_a3]


# Define the target function to fit using the analytical version.

def fitfunc(x, R, k, c0):
    return k/(1+c0*np.exp(-R*x))

def tau(x, R, k):
    return np.log(2)/(R*(1-x/k))

R_array_upper_growth2_upper_joint_analytical=[]
k_array_upper_growth2_upper_joint_analytical=[]
c0_array_upper_growth2_upper_joint_analytical=[]
tau_mean_upper_growth2_upper_joint_analytical=[]
tau_mean_1_upper_growth2_upper_joint_analytical=[]
tau_mean_2_upper_growth2_upper_joint_analytical=[]
tau_mean_3_upper_growth2_upper_joint_analytical=[]
for area in animals:

    # Fit the selected function to data.
    df = pd.read_excel(io=archivo, sheet_name=str(dataset))
    time = df["time"]
    Ca_data=[x for x in area if str(x) != 'nan']
    if len(Ca_data)<len(time):
        time.pop(len(time)-1)
    # test different initial guesses and select the one with higher R squared.
    p0s=[] 
    r_squared_array=[]
    for rep in range(4):
        for ranR in range(4):
            for rank in range(4):
                for ranc0 in range(4):
                    p0s.append([random.random()*10**ranR,random.random()*10**rank,random.random()*10**ranc0])
    for p0_test in p0s:
        k_fit2, kcov = curve_fit(fitfunc, time, Ca_data, bounds=([ 0, 0, -1], [np.inf, np.inf, np.inf]),p0=p0_test)
        residuals = Ca_data - fitfunc(time, *k_fit2)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((Ca_data-np.mean(Ca_data))**2)
        r_squared = 1 - (ss_res / ss_tot)
        r_squared_array.append(r_squared)
    best_p0=p0s[r_squared_array.index(max(r_squared_array))]
    k_fit, kcov = curve_fit(fitfunc, time, Ca_data, bounds=([ 0, 0, -1], [np.inf, np.inf, np.inf]),p0=best_p0)
    # append the fit results in arrays.
    R_array_upper_growth2_upper_joint_analytical.append(k_fit[0])
    k_array_upper_growth2_upper_joint_analytical.append(k_fit[1])
    c0_array_upper_growth2_upper_joint_analytical.append(k_fit[2])

    tfit = np.arange(0,max(time)+1,1);
    fit = fitfunc(tfit, k_fit[0], k_fit[1], k_fit[2])
    tau_plot = tau(fit, k_fit[0], k_fit[1])
    tau_mean_upper_growth2_upper_joint_analytical.append(np.mean(np.array(tau_plot)))
    before = []
    after = []
    arr1 = []
    arr2 = []
    arr3 = []
    for t in tfit:
        if t < max(time)/3:
            arr1.append(tau(fitfunc(t, k_fit[0], k_fit[1],k_fit[2]), k_fit[0], k_fit[1]))
        elif t < 2*max(time)/3 and t >= max(time)/3:
            arr2.append(tau(fitfunc(t, k_fit[0], k_fit[1],k_fit[2]), k_fit[0], k_fit[1]))
        else:
            arr3.append(tau(fitfunc(t, k_fit[0], k_fit[1],k_fit[2]), k_fit[0], k_fit[1]))
            
    tau_mean_1_upper_growth2_upper_joint_analytical.append(np.mean(np.array(arr1)))
    tau_mean_2_upper_growth2_upper_joint_analytical.append(np.mean(np.array(arr2)))
    tau_mean_3_upper_growth2_upper_joint_analytical.append(np.mean(np.array(arr3)))
    
R_array_upper_growth2_upper_joint_analytical=np.array(R_array_upper_growth2_upper_joint_analytical)
k_array_upper_growth2_upper_joint_analytical=np.array(k_array_upper_growth2_upper_joint_analytical)

# Create a .txt file with the results.  
with open("growth2-upper-joint-regeneration-upper.txt", "w") as file:
    file.write("R:\n")
    for line in R_array_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")
    
    file.write("k:\n")
    for line in k_array_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")
    
    file.write("tau_mean:\n")
    for line in tau_mean_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")
    
    file.write("tau_mean zone 1/3:\n")
    for line in tau_mean_1_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")
    
    file.write("tau_mean zone 2/3:\n")
    for line in tau_mean_2_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")
    
    file.write("tau_mean zone 3/3:\n")
    for line in tau_mean_3_upper_growth2_upper_joint_analytical:
        file.write(str(line)+ "\n")
    file.write("\n")